In [19]:
import pandas as pd
import torch 
from transformers import BertModel, BertForNextSentencePrediction, BertTokenizer, AdamW
from torch.nn import CrossEntropyLoss
from torch.nn.functional import nll_loss, kl_div

# Mock data

In [2]:
headline_1 = ["""Qualcomm's Win Over Apple in Germany Won't Force Apple to Table""",
"""Trump walks out of news conference after reporter asks him about Veterans Choice lie he's told more than 150 times""",
"""'Everything is gone.' Flooding in China ruins farmers and risks rising food prices"""]

body_1 = ["""Qualcomm's win over Apple in Germany, which if upheld will block the sale of some older model iPhones in Germany, could reduce Apple's quarterly sales by about $560 million, in our estimation. It's an important win for Qualcomm and builds on its momentum from a recent injunction in China, yet we don't think a victory in this battle will put enough pressure on Apple to settle the global dispute in the near term.""",
"""Trump, speaking at his Bedminster, New Jersey, golf club, had claimed again that he is the one who got the Veterans Choice program passed -- adding, "They've been trying to get that passed for decades and decades and decades and no president's ever been able to do it, and we got it done." In fact, former President Barack Obama signed the Choice program into law in 2014. The law, which allowed eligible veterans to be covered by the government for care provided by doctors outside the VA system, was a bipartisan initiative spearheaded by two senators Trump has repeatedly criticized, Bernie Sanders of Vermont and the late John McCain of Arizona. What Trump signed was a 2018 law, the VA MISSION Act, that modified and expanded the eligibility criteria from the Choice program. Rather than tout that bill, Trump has claimed over and over that he created Veterans Choice itself -- after others had failed for "50 years." """,
"""By this time of year, the rice growing on Bao Wentao's family farm should have been ready to harvest. Instead, heavy flooding has engulfed huge swathes of southern China, including more than 36 acres of rice fields that 19-year-old Bao and his father tend to in their village near Poyang Lake. "The crops have completely failed," Bao told CNN Business in an interview over the social media app WeChat, adding that his family has already lost roughly 200,000 yuan ($28,000) worth of produce. "The rice was nearly ripened and ready to harvest before the flooding. But now everything is gone." Surging floodwater burst the banks of Poyang Lake in Jiangxi province last month, destroying thousands of acres of farmland in what's known as the "land of fish and rice." The broader Yangtze River basin — which includes Poyang Lake and stretches more than 3,900 miles from Shanghai in the east to the Tibetan border in the west — accounts for 70% of the country's rice production."""]

headline_2 = ["""Qualcomm's Germany Offensive Coming Up Short in Apple Row""",
"""Former WWE wrestler James 'Kamala' Harris dies at 70""",
"""One of India's biggest food delivery companies has introduced period leave"""]

body_2 = ["""Qualcomm is losing steam in Germany with every decision that falls short of entirely blocking Apple's most popular iPhone models from being sold there. So far no judge has gone that far, with a Jan. 15 Mannheim court ruling in Apple's favor the latest setback for Qualcomm. The company sued Apple in Germany because the nation frequently grants broad injunctive relief in patent cases. We think Apple is likely to escape meaningful harm in Germany.""",
"""Harris, who performed in the ring as Kamala, made his WWF debut in 1984, according to WWE's website. In a career that spanned more than two decades, Harris battled many of wrestling's superstars, including Hulk Hogan, The Undertaker and Andre the Giant, according to the release. "He terrorized opponents and thrilled audiences in Mid-South, World Class Championship Wrestling, WCW and WWE until 2006," the WWE said. His character, Kamala, was promoted as a 6-foot-7, 350-pound "Ugandan Giant." A cause of death for Harris was not given. Other wrestlers took to social media Sunday to honor Harris using the hashtag #ripkamala.""",
"""In a statement Saturday, Zomato founder and CEO Deepinder Goyal said women and transgender people would be entitled to one day of period leave for each menstrual cycle. Zomato — which bought Uber's food delivery business in India earlier this year — is the most high profile company in the country to introduce period leave. At least a handful of other companies have introduced similar policies in the past, according to local media reports.
"At Zomato, we want to foster a culture of trust, truth and acceptance," Goyal said in an email to staff that was posted publicly. "There shouldn't be any shame or stigma attached to applying for a period leave. You should feel free to tell people on internal groups, or emails that you are on your period leave for the day." """]


In [3]:
a = zip(headline_1, body_1, headline_2, body_2)

data = pd.DataFrame([next(a), next(a), next(a)], columns=["HEADLINE_1", "BODY_1","HEADLINE_2","BODY_2"])

data.to_csv("train.csv")

In [4]:
data

,HEADLINE_1,BODY_1,HEADLINE_2,BODY_2
0,Qualcomm's Win Over Apple in Germany Won't For...,"Qualcomm's win over Apple in Germany, which if...",Qualcomm's Germany Offensive Coming Up Short i...,Qualcomm is losing steam in Germany with every...
1,Trump walks out of news conference after repor...,"Trump, speaking at his Bedminster, New Jersey,...",Former WWE wrestler James 'Kamala' Harris dies...,"Harris, who performed in the ring as Kamala, m..."
2,'Everything is gone.' Flooding in China ruins ...,"By this time of year, the rice growing on Bao ...",One of India's biggest food delivery companies...,"In a statement Saturday, Zomato founder and CE..."


In [6]:
aug_data = data.copy()
tmp = aug_data.iloc[-1,:][["HEADLINE_2","BODY_2"]].values
print(tmp)
aug_data.iloc[1:,:][["HEADLINE_2","BODY_2"]] = aug_data.iloc[:-1,:][["HEADLINE_2","BODY_2"]].values
print(aug_data.iloc[1:,:][["HEADLINE_2","BODY_2"]])
aug_data.iloc[0,:][["HEADLINE_2","BODY_2"]] = tmp
print(tmp)
print(aug_data.iloc[0,:][["HEADLINE_2","BODY_2"]])
aug_data["Label"] = 0

["One of India's biggest food delivery companies has introduced period leave"
 'In a statement Saturday, Zomato founder and CEO Deepinder Goyal said women and transgender people would be entitled to one day of period leave for each menstrual cycle. Zomato — which bought Uber\'s food delivery business in India earlier this year — is the most high profile company in the country to introduce period leave. At least a handful of other companies have introduced similar policies in the past, according to local media reports.\n"At Zomato, we want to foster a culture of trust, truth and acceptance," Goyal said in an email to staff that was posted publicly. "There shouldn\'t be any shame or stigma attached to applying for a period leave. You should feel free to tell people on internal groups, or emails that you are on your period leave for the day." ']
                                          HEADLINE_2  \
1  Qualcomm's Germany Offensive Coming Up Short i...   
2  Former WWE wrestler James 'Kam

In [7]:
data["Label"] = 1
aug_data["Label"] = 0

In [8]:
aug_data

,HEADLINE_1,BODY_1,HEADLINE_2,BODY_2,Label
0,Qualcomm's Win Over Apple in Germany Won't For...,"Qualcomm's win over Apple in Germany, which if...",One of India's biggest food delivery companies...,"In a statement Saturday, Zomato founder and CE...",0
1,Trump walks out of news conference after repor...,"Trump, speaking at his Bedminster, New Jersey,...",Qualcomm's Germany Offensive Coming Up Short i...,Qualcomm is losing steam in Germany with every...,0
2,'Everything is gone.' Flooding in China ruins ...,"By this time of year, the rice growing on Bao ...",Former WWE wrestler James 'Kamala' Harris dies...,"Harris, who performed in the ring as Kamala, m...",0


In [9]:
data

,HEADLINE_1,BODY_1,HEADLINE_2,BODY_2,Label
0,Qualcomm's Win Over Apple in Germany Won't For...,"Qualcomm's win over Apple in Germany, which if...",Qualcomm's Germany Offensive Coming Up Short i...,Qualcomm is losing steam in Germany with every...,1
1,Trump walks out of news conference after repor...,"Trump, speaking at his Bedminster, New Jersey,...",Former WWE wrestler James 'Kamala' Harris dies...,"Harris, who performed in the ring as Kamala, m...",1
2,'Everything is gone.' Flooding in China ruins ...,"By this time of year, the rice growing on Bao ...",One of India's biggest food delivery companies...,"In a statement Saturday, Zomato founder and CE...",1


In [10]:
pd.concat([data, aug_data]).reset_index(drop=True)

,HEADLINE_1,BODY_1,HEADLINE_2,BODY_2,Label
0,Qualcomm's Win Over Apple in Germany Won't For...,"Qualcomm's win over Apple in Germany, which if...",Qualcomm's Germany Offensive Coming Up Short i...,Qualcomm is losing steam in Germany with every...,1
1,Trump walks out of news conference after repor...,"Trump, speaking at his Bedminster, New Jersey,...",Former WWE wrestler James 'Kamala' Harris dies...,"Harris, who performed in the ring as Kamala, m...",1
2,'Everything is gone.' Flooding in China ruins ...,"By this time of year, the rice growing on Bao ...",One of India's biggest food delivery companies...,"In a statement Saturday, Zomato founder and CE...",1
3,Qualcomm's Win Over Apple in Germany Won't For...,"Qualcomm's win over Apple in Germany, which if...",One of India's biggest food delivery companies...,"In a statement Saturday, Zomato founder and CE...",0
4,Trump walks out of news conference after repor...,"Trump, speaking at his Bedminster, New Jersey,...",Qualcomm's Germany Offensive Coming Up Short i...,Qualcomm is losing steam in Germany with every...,0
5,'Everything is gone.' Flooding in China ruins ...,"By this time of year, the rice growing on Bao ...",Former WWE wrestler James 'Kamala' Harris dies...,"Harris, who performed in the ring as Kamala, m...",0


In [11]:
data = pd.read_csv("./augmented_data.csv", index_col=0)
data = data.values

In [12]:
data

array([["Qualcomm's Win Over Apple in Germany Won't Force Apple to Table",
        "Qualcomm's win over Apple in Germany, which if upheld will block the sale of some older model iPhones in Germany, could reduce Apple's quarterly sales by about $560 million, in our estimation. It's an important win for Qualcomm and builds on its momentum from a recent injunction in China, yet we don't think a victory in this battle will put enough pressure on Apple to settle the global dispute in the near term.",
        "Qualcomm's Germany Offensive Coming Up Short in Apple Row",
        "Qualcomm is losing steam in Germany with every decision that falls short of entirely blocking Apple's most popular iPhone models from being sold there. So far no judge has gone that far, with a Jan. 15 Mannheim court ruling in Apple's favor the latest setback for Qualcomm. The company sued Apple in Germany because the nation frequently grants broad injunctive relief in patent cases. We think Apple is likely to escape 

# NSP module

In [13]:
class NSP(torch.nn.Module):
    def __init__(self, model):
        super(NSP, self).__init__()
        self.model = model

        # Pool hidden layers corresponding to the first token of the input sequence ([CLS])

        self.tanh = torch.nn.Tanh()
        self.pool = torch.nn.Linear(self.model.config.hidden_size, self.model.config.hidden_size)

        self.seq_prediction = torch.nn.Linear(self.model.config.hidden_size, 2)

    def forward(self, input_ids, token_type_ids, attention_mask, cls_pos = 0):
        # the position of CLS being encoded is different for different models
        # cls_pos = -1 : encoded at the end of the sequence (eg. XLNet)
        # cls_pos = 0 : encoded at the beginning of the sequence (eg. BERT)
        output = self.model(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask
        )

        pooled_output = self.pool(self.tanh(output[0][:,cls_pos]))

        return self.seq_prediction(pooled_output)


In [15]:
bertmodel = BertModel.from_pretrained("bert-base-uncased")
bertnsp = BertForNextSentencePrediction.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
model = NSP(bertmodel)

calc_loss = CrossEntropyLoss()

optimizer = AdamW(model.parameters(), lr = 0.0001)

In [22]:
n = len(data)

for sample in data:
    inp = [sample[0]+" "+tokenizer.pad_token+" "+sample[1], 
           sample[2]+" "+tokenizer.pad_token+" "+sample[3]]
    tar = torch.tensor([sample[-1]]).type(torch.long)
    tokenized = tokenizer(inp[0], inp[1], 
                          return_tensors="pt", 
                          max_length=512, 
                          truncation=True, 
                          padding="max_length"
                          )
    output = model(**tokenized)
    loss = CrossEntropyLoss()(output,tar)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

In [30]:
tokenizer.decode(tokenizer(sample[0],sample[1],sample[2])["input_ids"])

'[CLS]\'everything is gone.\'flooding in china ruins farmers and risks rising food prices [SEP] by this time of year, the rice growing on bao wentao\'s family farm should have been ready to harvest. instead, heavy flooding has engulfed huge swathes of southern china, including more than 36 acres of rice fields that 19 - year - old bao and his father tend to in their village near poyang lake. " the crops have completely failed, " bao told cnn business in an interview over the social media app wechat, adding that his family has already lost roughly 200, 000 yuan ( $ 28, 000 ) worth of produce. " the rice was nearly ripened and ready to harvest before the flooding. but now everything is gone. " surging floodwater burst the banks of poyang lake in jiangxi province last month, destroying thousands of acres of farmland in what\'s known as the " land of fish and rice. " the broader yangtze river basin — which includes poyang lake and stretches more than 3, 900 miles from shanghai in the east 

"Former WWE wrestler James 'Kamala' Harris dies at 70"